![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20DeBertaForQuestionAnswering.ipynb)

## Import DeBertaForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 4.0.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import DeBERTa v2 & v3 models trained/fine-tuned for question answering via `DeBertaForQuestionAnswering` or `TFDeBertaForQuestionAnswering`. These models are usually under `Question Answering` category and have `deberta-v2` or `deberta-v3` in their labels
- Reference: [TFDebertaV2ForQuestionAnswering](https://huggingface.co/docs/transformers/model_doc/deberta-v2#transformers.TFDebertaV2ForQuestionAnswering)
- Some [example models](https://huggingface.co/models?filter=deberta-v2&pipeline_tag=question-answering)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.25.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- DeBERTa v2&v3 use SentencePiece, so we will have to install that as well


In [ ]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0 sentencepiece

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [nbroad/deberta-v3-xsmall-squad2](https://huggingface.co/nbroad/deberta-v3-xsmall-squad2) model from HuggingFace as an example
- In addition to `TFDebertaV2ForQuestionAnswering` we also need to save the `DebertaV2Tokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from transformers import TFDebertaV2ForQuestionAnswering, DebertaV2Tokenizer 
import tensorflow as tf

MODEL_NAME = 'nbroad/deberta-v3-xsmall-squad2'

tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

try:
  model = TFDebertaV2ForQuestionAnswering.from_pretrained(MODEL_NAME)
except:
  model = TFDebertaV2ForQuestionAnswering.from_pretrained(MODEL_NAME, from_pt=True)
    
# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask"),
          "token_type_ids": tf.TensorSpec((None, None), tf.int32, name="token_type_ids"),
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})


Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

total 552808
-rw-r--r--  1 maziyar  staff        880 Dec 15 14:38 config.json
drwxr-xr-x  3 maziyar  staff         96 Dec 15 14:38 saved_model
-rw-r--r--  1 maziyar  staff  283030336 Dec 15 14:38 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 49384
drwxr-xr-x  2 maziyar  staff        64 Dec 15 14:38 assets
-rw-r--r--  1 maziyar  staff        54 Dec 15 14:38 fingerprint.pb
-rw-r--r--  1 maziyar  staff    177566 Dec 15 14:38 keras_metadata.pb
-rw-r--r--  1 maziyar  staff  25097793 Dec 15 14:38 saved_model.pb
drwxr-xr-x  4 maziyar  staff       128 Dec 15 14:38 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 4840
-rw-r--r--  1 maziyar  staff       23 Dec 15 14:36 added_tokens.json
-rw-r--r--  1 maziyar  staff      173 Dec 15 14:36 special_tokens_map.json
-rw-r--r--  1 maziyar  staff  2464616 Dec 15 14:36 spm.model
-rw-r--r--  1 maziyar  staff      486 Dec 15 14:36 tokenizer_config.json


- As you can see, we need the SavedModel from `saved_model/1/` path
- We also be needing `spm.model` from the tokenizer
- All we need is to just copy the `spm.model` to `saved_model/1/assets` which Spark NLP will look for

In [ ]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

!cp {MODEL_NAME}_tokenizer/spm.model {asset_path}

Voila! We have our `spm.model` inside assets directory

In [ ]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 4816
-rw-r--r--  1 maziyar  staff  2464616 Dec 15 14:38 spm.model


## Import and Save DeBertaForQuestionAnswering in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.1 and Spark NLP 4.2.4
setup Colab for PySpark 3.2.1 and Spark NLP 4.2.4


Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `DeBertaForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DeBertaForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

spanClassifier = DeBertaForQuestionAnswering.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
spanClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your DeBertaForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 605592
-rw-r--r--  1 maziyar  staff  307593450 Dec 15 14:38 deberta_classification_tensorflow
-rw-r--r--  1 maziyar  staff    2464616 Dec 15 14:38 deberta_spp
drwxr-xr-x  3 maziyar  staff         96 Dec 15 14:38 fields
drwxr-xr-x  6 maziyar  staff        192 Dec 15 14:38 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny DeBertaForQuestionAnswering model in Spark NLP 🚀 pipeline! 

In [ ]:
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = DeBertaForQuestionAnswering.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

example = spark.createDataFrame([["What's my name?", "My name is Clara and I live in Berkeley."]]).toDF("question", "context")
result = pipeline.fit(example).transform(example)

result.select("answer.result").show(1, False)

+-------+
|result |
+-------+
|[Clara]|
+-------+



That's it! You can now go wild and use hundreds of `DeBertaForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀 
